# What makes an NBA All-Star?
### STAT 206 Final Project

#### Matthew Barclay
#### Riley Baumgarten
#### Arvind Kamboh

In [ ]:
import Pkg
Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

In [ ]:
using CSV, DataFrames
using Plots, StatsPlots
using Distributions
using MLJ, MLJClusteringInterface
using NearestNeighbors, StableRNGs, Random
#import PlotlyJS as PJS#import because PlotlyJS overwrites all plots and statsplots
#using Colors
using MLJLIBSVMInterface
using FreqTables

## Introduction: 

In the landscape of professional basketball, achieving the status of an NBA All-Star represents a significant milestone in a basketball player’s career, symbolizing not only individual excellence but also recognition among peers and fans. Nevertheless, the process through which NBA players attain this distinction remains a multifaceted and often debated topic. This analysis project aims to investigate the determinants that influence the selection of NBA players as All-Stars. By examining factors such as point per game, minutes played, rebounds, and shot percentage, this project seeks to explore the dynamics shaping the All-Star selection process. 

## Data Summary: 

For this project, we used the NBA Player Stats Dataset for the 2022-2023 season, obtained from Kaggle. The actual data comes from the NBA statistical source Basketball Reference. This dataset comprises 500 rows and 30 columns, with each row representing an NBA player and each column denoting various statistical attributes such as points per game, assists, rebounds, and shooting percentages, among others. This dataset was augmented with outside data to include an additional column indicating whether a player has been selected as an All-Star in previous seasons, which provides valuable historical context for our analysis to distinguish the current players who were All-Stars. This dataset contains no missing values and presents a solid foundation for addressing our research question regarding the determinants influencing an NBA player's selection as an All-Star, given its comprehensive coverage of accurate player statistics and the NBA being such a highly data-driven organization.
Source: https://www.kaggle.com/datasets/bryanchungweather/nba-players-data-2022-2023  

Columns Description:
1. Rk: Rank
2. Player: Player's name
3. Pos: Position
4. Age: Player's age
5. Tm: Team
6. G: Games played
7. GSpct: Games started percentage
8. MP: Minutes played per game
9. FG: Field goals per game
10. FGA: Field goal attempts per game
11. FGpct: Field goal percentage
12. ThreeP: 3-point field goals per game
13. ThreePA: 3-point field goal attempts per game
14. ThreePpct: 3-point field goal percentage
15. TwoP: 2-point field goals per game
16. TwoPA: 2-point field goal attempts per game
17. TwoPpct: 2-point field goal percentage
18. eFGpct: Effective field goal percentage
19. FT: Free throws per game
20. FTA: Free throw attempts per game
21. FTpct: Free throw percentage
22. ORB: Offensive rebounds per game
23. DRB: Defensive rebounds per game
24. TRB: Total rebounds per game
25. AST: Assists per game
26. STL: Steals per game
27. BLK: Blocks per game
28. TOV: Turnovers per game
29. PF: Personal fouls per game (committed)
30. PPG: Points per game
31. ALLSTAR: All-Star status

In [ ]:
nbadf = CSV.read("nba_2022_2023.csv", DataFrame, stringtype = String) |> dropmissing
#describe(nbadf)
rename!(nbadf,
Symbol("FG%")=>:FGpct,
Symbol("3P")=>:ThreeP,
Symbol("3PA")=>:ThreePA,
Symbol("3P%")=>:ThreePpct,
Symbol("2P")=>:TwoP,
Symbol("2PA")=>:TwoPA,
Symbol("2P%")=>:TwoPpct,
Symbol("eFG%")=>:eFGpct,
Symbol("FT%")=>:FTpct,
:PTS=>:PPG
)
multiteam = filter(:Tm=>==("TOT"),nbadf).Player #identify players on multiple teams
filter!(row -> row.Tm==("TOT") || !in(row.Player, multiteam),nbadf) #remove all rows but total for players on multiple teams
#Convert multi-positional to their main position
nbadf.Pos = first.(nbadf.Pos,2)
#filter(:Pos => in(["SG-PG","SF-SG","PG-SG","SF-PF","PF-C","PF-SF"]), nbadf)

allstars = CSV.read("nba_2022_2023_allstars.csv", DataFrame, stringtype = String)
leftjoin!(nbadf,allstars, on = :Player => :NAME)
replace!(nbadf.ALLSTAR, missing => "N")
#filter(:ALLSTAR=>==("Y"),nbadf)
nbadf.GSpct = nbadf.GS ./ nbadf.G
describe(nbadf)

## Research Question: 

> What are the determinant factors influencing an NBA player's selection as an All-Star?

## Methods: 

In order to solve our research question, there are a few key methods that are vital in helping us analyze the data we were given:

*Exploratory Data Analysis (EDA)*

+ **Correlation Matrices:** To identify potential relationships between variables and remove predictors with extreme correlation values.

+ **Histograms and Boxplots:** These will be used to visualize the distribution of key variables and identify outliers or skewness in the data, providing insights into the data's characteristics.

+ **Principal Component Analysis (PCA):** PCA will be applied to reduce the dimensionality of the dataset, identifying a smaller number of uncorrelated variables (principal components) that capture most of the variance in the data.

*Predictive Modeling*

+ **Logistic Regression:** A baseline model due to its interpretability and effectiveness in binary classification tasks.
Decision Trees and Random Forests: These models are useful for handling nonlinear relationships and interactions between variables.

+ **K-Nearest Neighbors (KNN):** To explore the predictive power of player similarities in the context of All-Star selections.

+ **Linear Discriminant Analysis (LDA):** As a technique to find a linear combination of features that characterizes or separates two or more classes (All-Star selected/not selected).

+ **Neural Networks:** To model complex nonlinear relationships through a deep learning approach, potentially capturing intricate patterns in the data.

+ **XGBoost:** A gradient boosting framework that has been successful in various prediction tasks, known for its performance and speed.

*Addressing Potential Issues*

+ **Multicollinearity:** Before deploying logistic regression and other linear models, the correlation will be calculated and the raw data sources analyzed to identify and mitigate multicollinearity among predictors.

+ **Overfitting:** Regularization techniques, along with validation methods such as holdout-validation, will be applied to prevent overfitting, especially in complex models like neural networks and random forests.

By employing these methods, the determinants of NBA All-Star can be selected accurately, providing both predictive power and insights into the importance and influence of various factors.


## Results

### Correlation and Multicollinearity

In [ ]:
numeric_df = select(nbadf, Not([:Player, :Pos, :Tm, :ALLSTAR,:GS]))
vars = names(numeric_df)
X = Matrix(numeric_df)
M = cor(X)
fig = heatmap(M, 
    title="Correlation Matrix - All Predictors", 
    xticks=(1:26, vars), 
    yticks=(1:26, vars), 
    clims=(-1,1), 
    xrot=45,
    color=cgrad(:balance, rev=true), 
    aspect=:ratio, 
    size=(900, 800)
)

for j in axes(M,2), i in axes(M,1)
    annotate!(i,j, text("$(round(M[i,j], digits=2))", :white, 8))
end 
fig 

### Check for multicollinearity

The following predictors were removed from the dataset due to high correlation and issues with multicollinearity:
+ Shot attempts: high correlation between shots made and attempts, better to use shots made and shot pct
+ eFGpct: According to [basketball-reference.com](https://www.basketball-reference.com/about/glossary.html), eFGpct = (FG + 0.5 * 3P) / FGA which is a linear equation of predictors
+ Field Goals: Field goals is just 2P + 3P, so it is collinear
+ Total Rebounds: Total rebounds is offensive + defensive rebounds, which is collinear
+ Recommendation: Remove these predictors when creating linear models

In [ ]:
vars = names(select(numeric_df, Not([:FG,:FGA,:FGpct,:ThreePA,:TwoPA,:FTA,:TRB,:eFGpct, :PPG])))
X = Matrix(select(numeric_df, Not([:FG,:FGA,:FGpct,:ThreePA,:TwoPA,:FTA,:TRB,:eFGpct, :PPG])))
M = cor(X)
fig = heatmap(M, 
    title="Correlation Matrix", 
    xticks=(1:17, vars), 
    yticks=(1:17, vars), 
    clims=(-1,1), 
    xrot=45,
    color=cgrad(:balance, rev=true), 
    aspect=:ratio, 
    size=(900, 800)
)

for j in axes(M,2), i in axes(M,1)
    annotate!(i,j, text("$(round(M[i,j], digits=2))", :white, 8))
end 
fig 

While there are still a few predictors with high correlation, the overall correlation looks much better and the concerns of multicollinearity have been removed.

##  Data Visualization

In [ ]:
# To make a ColorGradient for Heatmaps: color_palette = cgrad(Sample color scheme = :roma, scale = :log))
@df nbadf scatter(:MP, :PPG, group = :ALLSTAR, 
smooth = true, linewidth = 4,
title = "Playtime efficiency",
xlabel = "Minutes played per game",
ylabel = "Points per game",
label = ["Players" "Allstars"])

The scatterplot of minutes played against points per game appears to be non-linear until the Allstar status is labeled on the points. Then we see that there is not as much of a non-linear relationship but actually two different linear relationships, where Allstars play more minutes and have a higher rate of scoring points during their playing time.

In [ ]:
filter(row->row.ALLSTAR==("Y") && row.MP<25,nbadf)

While most Allstars had high playing time, there appeared to be a few outliers who played less than 25 minutes per game. These players in general were dealing with injuries or older age which limited their play time during this season.

In [ ]:
groupedhist(nbadf.Age, group=nbadf.ALLSTAR, bar_position = :stack,
title = "Player Age Distribution",
xlabel = "Age",
ylabel = "Count",
label = ["Players" "Allstars"])

In [ ]:
@df nbadf scatter(:ThreePA, :TwoPA, group = :ALLSTAR,
title = "Shot Attempts Per Game",
xlabel = "Three Point Attempts",
ylabel = "Two Point Attempts",
label = ["Players" "Allstars"])

In [ ]:
boxplot(nbadf.Age, nbadf.G, xlabel="Age", ylabel="Games Played", label= nothing, 
size=(600, 300))


## PCA Analysis

In [ ]:
features = names(numeric_df)
# load the PCA model
PCA = @load PCA pkg = MultivariateStats

# Standarize -> PCA w/ 97.5% variance.
# The mean = 0 option means we do not need to center the data.
model = Pipeline(Standardizer(), PCA(variance_ratio = 0.975))

# Fit the machine.
mach = machine(model, numeric_df) |> MLJ.fit!

# Apply a transformation to the numeric data and convert to Matrix.
Xproj = MLJ.transform(mach, numeric_df) |> Matrix
;

In [ ]:
r = report(mach).pca

In [ ]:
y = nbadf[:, :ALLSTAR] |> Vector
scatter(Xproj[:, 1], Xproj[:, 2],
  group = y,
  title = "NBA Data in PC coordinates",
  xlabel = "PC1",
  ylabel = "PC2",
  label = ["Players" "Allstars"]
)

In [ ]:
y = nbadf[:, :ALLSTAR] |> Vector
scatter(Xproj[:, 1], Xproj[:, 2], Xproj[:, 3],
  group = y,
  title = "NBA Data in PC coordinates",
  xlabel = "PC1",
  ylabel = "PC2",
  zlabel = "PC3",
  label = ["Players" "Allstars"]
)

In [ ]:
L = r.loadings
loadings_df = hcat(
  DataFrame(feature = features),
  DataFrame(L, :auto)
)

rename!(loadings_df, [:feature, :PC1, :PC2, :PC3])
loadings_df

#### Interactive scatterplot from PlotlyJS

In [ ]:
# PCA_df = DataFrame(:ALLSTAR=>nbadf[:,:ALLSTAR],:X1=>Xproj[:, 1],:X2=>Xproj[:, 2],:X3=>Xproj[:,3])

# colors = [RGB(18/255, 133/255, 248/255), RGB(217/255, 89/255, 56/255)]
# data = PJS.GenericTrace[]
# df = filter(:ALLSTAR=>==("N"),PCA_df)
# PC1=df[:,:X1]
# PC2=df[:,:X2]
# PC3=df[:,:X3]
# trace = PJS.scatter3d(name="Players", mode="markers",
#                     marker_size=3, marker_color=colors[1], marker_line_width=0,
#                     x=PC1, y=PC2, z=PC3)
# push!(data, trace)
# cluster = PJS.mesh3d(color=colors[1], opacity=0.3, x=PC1, y=PC2, z=PC3)
# push!(data, cluster)

# df = filter(:ALLSTAR=>==("Y"),PCA_df)
# PC1=df[:,:X1]
# PC2=df[:,:X2]
# PC3=df[:,:X3]
# trace = PJS.scatter3d(name="Allstars", mode="markers",
#                     marker_size=3, marker_color=colors[2], marker_line_width=0,
#                     x=PC1, y=PC2, z=PC3)
# push!(data, trace)
# cluster = PJS.mesh3d(color=colors[2], opacity=0.3, x=PC1, y=PC2, z=PC3)
# push!(data, cluster)
# # notice the nested attrs to create complex JSON objects
# layout = PJS.Layout(width=800, height=550, autosize=false, title="Interactive NBA Data in PC coordinates",
#                 scene=PJS.attr(xaxis=PJS.attr(gridcolor="rgb(255, 255, 255)",
#                                         zerolinecolor="rgb(255, 255, 255)",
#                                         showbackground=true,
#                                         backgroundcolor="rgb(230, 230,230)",
#                                         title = "PC1"),
#                             yaxis=PJS.attr(gridcolor="rgb(255, 255, 255)",
#                                         zerolinecolor="rgb(255, 255, 255)",
#                                         showbackground=true,
#                                         backgroundcolor="rgb(230, 230,230)",
#                                         title = "PC2"),
#                             zaxis=PJS.attr(gridcolor="rgb(255, 255, 255)",
#                                         zerolinecolor="rgb(255, 255, 255)",
#                                         showbackground=true,
#                                         backgroundcolor="rgb(230, 230,230)",
#                                         title = "PC3"),
#                             aspectratio=PJS.attr(x=1, y=1, z=0.7),
#                             aspectmode = "manual"),
#                             scene_camera = PJS.attr(eye=PJS.attr(x=1.25, y=-2, z=.75) # Try to match Plots output
#                             ))
# # p = PJS.plot(data, layout)
# # open("./PCA3D.html", "w") do io
# #     PJS.PlotlyBase.to_html(io, p.plot)
# # end

<iframe width=900 height=600 src="./PCA3D.html"></iframe>

# Classification Models

In [ ]:
y = categorical(String.(nbadf[!, :ALLSTAR]), levels = ["N", "Y"])
X = select(numeric_df, Not([:FG,:FGA,:FGpct,:ThreePA,:TwoPA,:FTA,:TRB,:eFGpct, :PPG]))
X.Age2 = X.Age.^2 
X.Pos = categorical(nbadf.Pos) 

#Code from Lecture 19
train, test = partition(eachindex(y), 0.8, shuffle=true, rng=206);
acc = Float64[] # accuracy()
pre = Float64[] # multiclass_precision()
rec = Float64[] # multiclass_recall()
f1s = Float64[] # f1score()
mat = []        # confusion_matrix()

In [ ]:
#Code from Lecture 19
KNNClassifier = @load KNNClassifier verbosity = 0
LDA = @load LDA verbosity = 0
NeuralNetworkClassifier = @load NeuralNetworkClassifier pkg = MLJFlux verbosity = 0
#MultinomialClassifier = @load MultinomialClassifier verbosity = 0 #not needed, same as logistic when only 2 classes
DecisionTreeClassifier = @load DecisionTreeClassifier pkg = DecisionTree verbosity=0
RandomForestClassifier = @load RandomForestClassifier pkg=DecisionTree verbosity=0
LogisticClassifier = @load LogisticClassifier pkg= MLJLinearModels verbosity=0
XGBoostClassifier = @load XGBoostClassifier pkg = XGBoost verbosity=0
model_list = [
    KNNClassifier(K = 5), # use nearest 5-neighbors to make predictions
    LDA(),
    NeuralNetworkClassifier(epochs = 20),
    #MultinomialClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticClassifier(),
    XGBoostClassifier()
]

In [ ]:
#Code from Lecture 19
for clf in model_list
    Random.seed!(206)
    # Create a pipeline model that standardizes, then fits a classifier.
    model = Pipeline(Standardizer(), 
                        OneHotEncoder(),
                        clf)
    # Fit the model onto the training set
    mach = machine(model, X, y)
    fit!(mach, rows = train, verbosity = 0)
    # Make predictions on the test set
    yhat = MLJ.predict(mach, rows = test)
    # Evaluate the model on the test set using selected metrics
    #
    # NOTES:
    #
    #   - MLJ.predict() may give probabilistic predictions. Use mode() to collapse to a concrete target.
    #   - An evaluation metric F() always accepts inputs as F(fitted, observed).
    #
    push!(acc, accuracy(mode.(yhat), y[test]))
    push!(pre, multiclass_precision(mode.(yhat), y[test]))
    push!(rec, multiclass_recall(mode.(yhat), y[test]))
    push!(f1s, f1score(mode.(yhat), y[test]))
    push!(mat, ConfusionMatrix(levels = levels(y))(mode.(yhat), y[test]))
end

In [ ]:
#Code from Lecture 19
results = DataFrame(
    Model = typeof.(model_list),
    Accuracy = acc,
    Precision = pre,
    Recall = rec,
    F1 = f1s
)

#### KNN

In [ ]:
mat[1]  # KNNClassifier

#### LDA

In [ ]:
mat[2]  # LDA

#### Neural Network

In [ ]:
mat[3]  # NeuralNetworkClassifier

#### Decision Tree

In [ ]:
mat[4] #Decision Tree

#### Ridge Regression

In [ ]:
mat[5] #Ridge

#### Logistic Regression

In [ ]:
mat[6] #Logistic

#### XGBoost

In [ ]:
mat[7] #XGBoost

In [ ]:
model = Pipeline(Standardizer(), OneHotEncoder(), LogisticClassifier())
    # Fit the model onto the training set
    mach = machine(model, X, y_encode)
    fit!(mach, verbosity = 0)
fitted_params(mach).logistic_classifier.coefs |> DataFrame
#MLJ.predict(mach)